In [ ]:
#!/usr/bin/env python
""" 
Python script to download selected files from rda.ucar.edu.
After you save the file, don't forget to make it executable
i.e. - "chmod 755 <name_of_script>"
"""
import sys, os
from urllib.request import build_opener

opener = build_opener()

filelist = [
  'https://request.rda.ucar.edu/dsrqst/WANG761241/TarFiles/gfs.0p25.2021092400.f384-25.2021101100.f000.grib2.nc.tar'
]

for file in filelist:
    ofile = os.path.basename(file)
    sys.stdout.write("downloading " + ofile + " ... ")
    sys.stdout.flush()
    infile = opener.open(file)
    outfile = open(ofile, "wb")
    outfile.write(infile.read())
    outfile.close()
    sys.stdout.write("done\n")


In [2]:
import tarfile

# 指定要解压缩的文件路径
file_path = 'PVnet_data_hk/nwp/gfs.0p25.2021092400.f384-25.2021101100.f000.grib2.nc.tar'

# 使用tarfile模块打开文件
with tarfile.open(file_path, 'r:*') as tar:
    # 列出tar文件中的内容
    print(tar.getnames())
    # 解压缩所有文件到指定目录
    tar.extractall(path='./raw2/')

# 如果文件是.tar.gz或.tar.bz2格式，你可以将模式更改为相应的'gz'或'bz2'

FileNotFoundError: [Errno 2] No such file or directory: 'PVnet_data_hk/nwp/gfs.0p25.2021092400.f384-25.2021101100.f000.grib2.nc.tar'

In [ ]:
import zipfile
import os

# 指定包含zip文件的目录
zip_folder_path = './raw2/'
# 指定解压后的文件存放目录
extract_to_path = './raw3/'

# 确保解压目录存在，如果不存在则创建
if not os.path.exists(extract_to_path):
    os.makedirs(extract_to_path)

# 遍历指定目录下的所有文件
for filename in os.listdir(zip_folder_path):
    if filename.endswith('.zip'):  # 检查文件扩展名是否为.zip
        # 构造完整的zip文件路径
        zip_file_path = os.path.join(zip_folder_path, filename)
        # 使用with语句确保zip文件被正确关闭
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            # 解压缩文件到指定目录
            zip_ref.extractall(extract_to_path)
            print(f"已解压：{filename}")

print(f"所有文件已解压到：{extract_to_path}")

In [3]:
import os

nc_dir = './raw3/'
nc_file_list = os.listdir(nc_dir)

import xarray as xr
timelist = []
# 指定NetCDF文件的路径
for nc_file in nc_file_list:
    if 'f000' in nc_file:
        file_path = nc_dir+nc_file  
        dataset = xr.open_dataset(file_path)
        try:
            time = dataset.time0.values[0]
        except:
            time = dataset.time.values[0]
        print(time)
        timelist.append(time)


2021-10-10T00:00:00.000000000
2021-10-10T18:00:00.000000000
2021-10-10T06:00:00.000000000
2021-10-10T12:00:00.000000000


In [4]:
sorted(timelist)


[np.datetime64('2021-10-10T00:00:00.000000000'),
 np.datetime64('2021-10-10T06:00:00.000000000'),
 np.datetime64('2021-10-10T12:00:00.000000000'),
 np.datetime64('2021-10-10T18:00:00.000000000')]

In [5]:
import os
import pandas as pd
import xarray as xr
import numpy as np
nc_dir = './raw3/'
nc_file_list = os.listdir(nc_dir)


# 指定NetCDF文件的路径
file_path = nc_dir+nc_file_list[0]  # 替换为你的NetCDF文件路径

# 使用xarray打开NetCDF文件
dataset = xr.open_dataset(file_path)
dataset
 

<xarray.Dataset> Size: 25kB
Dimensions:                 (time0: 1, time1: 1, ntb: 2, level0: 41,
                             level1: 22, lat: 9, lon: 9)
Coordinates:
  * time0                   (time0) datetime64[ns] 8B 2021-10-10T12:00:00
  * time1                   (time1) datetime64[ns] 8B 2021-10-10T12:00:00
  * level0                  (level0) float32 164B 1e+03 975.0 ... 0.02 0.01
  * level1                  (level1) float32 88B 1e+03 975.0 ... 100.0 50.0
  * lat                     (lat) float32 36B 23.0 22.75 22.5 ... 21.25 21.0
  * lon                     (lon) float32 36B 113.0 113.2 113.5 ... 114.8 115.0
Dimensions without coordinates: ntb
Data variables: (12/22)
    valid_date_time0        (time0) |S10 10B ...
    ref_date_time0          (time0) |S10 10B ...
    forecast_hour0          (time0) timedelta64[ns] 8B ...
    time_bnds1              (time1, ntb) datetime64[ns] 16B ...
    valid_date_time_range1  (time1, ntb) |S10 20B ...
    ref_date_time1          (time1) |S10 10B ...
    ...                      ...
    M_CDC_L224_Avg_1        (time1, lat, lon) float32 324B ...
    M_CDC_L224              (time0, lat, lon) float32 324B ...
    H_CDC_L234_Avg_1        (time1, lat, lon) float32 324B ...
    H_CDC_L234              (time0, lat, lon) float32 324B ...
    4LFTX_L1                (time0, lat, lon) float32 324B ...
    ALBDO_L1_Avg_1          (time1, lat, lon) float32 324B ...
Attributes:
    Creation date and time:  2024-10-09 02:39:50 -0600
    Conventions:             CF-1.5
    Creator:                 NCAR - CISL RDA (dattore)

In [6]:
# 根据文件名推断实际预测时间点的函数

def find_real_time(filename):
    from datetime import datetime, timedelta
    filename_split = filename.split('.')
    init_time = filename_split[2]
    step_in_hour = int(filename_split[3][1:])
    # 步骤1：将日期时间字符串转换为datetime对象
    date_format = '%Y%m%d%H'
    dt = datetime.strptime(init_time, date_format)

    # 步骤2：将300小时转换为timedelta对象
    delta = timedelta(hours=step_in_hour)

    # 步骤3：将timedelta对象加到datetime对象上
    new_dt = dt + delta

    # 步骤4：将结果转换回所需的格式
    new_date_str = new_dt.strftime(date_format)

    return new_date_str


def paths_to_datetimeindex(paths,deltadays=0):
    date_strings = [i.split('.')[2] for i in paths]
    return pd.to_datetime(date_strings, format='%Y%m%d%H') + pd.Timedelta(days=deltadays)
find_real_time(nc_file_list[0])    
    
 


'2021101012'

In [7]:
#遍历所有文件，统计到底有几个目标时间
target_time_list = []
nc_dir = './raw3/'
nc_file_list = os.listdir(nc_dir)
for nc_file in nc_file_list:
    target_time = find_real_time(nc_file)   
    if target_time not in target_time_list:
        target_time_list.append(target_time)
target_time_list = sorted(target_time_list)
#target_time_list        

In [19]:
target_time = target_time_list[0]
matched_files = []
for nc_file in nc_file_list:
    if find_real_time(nc_file) == target_time:
        matched_files.append(nc_file)
matched_files = sorted(matched_files)



# Load in and concatenate all  matched nc files
xr_list = []
matched_files_2 = []
for matched_file_path_ in matched_files:
    xr_temp = xr.open_dataset(nc_dir+matched_file_path_)
    if "time0" not in xr_temp.dims :
        continue
    xr_list.append(xr_temp)
    matched_files_2.append(matched_file_path_)
# Create variable used for time axis
time_var = xr.Variable('init_time_utc', paths_to_datetimeindex(matched_files_2,deltadays=2))
    
geotiffs_da = xr.concat(xr_list,  dim=time_var)
# 创建一个'step'维度
time_selta_step_list= np.array([int(i.split('.')[3][1:])*3600*1e9 for i in matched_files_2])[::-1]
time_selta_step_list= time_selta_step_list-time_selta_step_list.min() #让step的第一个是0
timedeltas = np.array(time_selta_step_list, dtype='timedelta64[ns]')
new_step_dim = xr.DataArray(timedeltas, dims='step')

# 添加到数据集
ds_with_step = geotiffs_da.expand_dims({'step': new_step_dim})
del geotiffs_da

ds_with_step = ds_with_step.rename({"level1": "isobaricInhPa"})
ds_with_step = ds_with_step.rename({"lat": "latitude"})
ds_with_step = ds_with_step.rename({"lon": "longitude"})

ds_with_step =  ds_with_step.transpose("init_time_utc", "step", "latitude", "longitude",...)
ds_with_step.to_zarr('./final/gfs.0p25.'+target_time+'v2.zarr')
ds_with_step


<xarray.Dataset> Size: 67MB
Dimensions:                 (step: 52, init_time_utc: 52, time0: 1, time1: 1,
                             ntb: 2, latitude: 9, longitude: 9, level0: 41,
                             isobaricInhPa: 22)
Coordinates:
  * step                    (step) timedelta64[ns] 416B 0 days 06:00:00 ... 1...
  * time0                   (time0) datetime64[ns] 8B 2021-10-10
  * time1                   (time1) datetime64[ns] 8B 2021-10-10
  * level0                  (level0) float32 164B 1e+03 975.0 ... 0.02 0.01
  * isobaricInhPa           (isobaricInhPa) float32 88B 1e+03 975.0 ... 50.0
  * latitude                (latitude) float32 36B 23.0 22.75 ... 21.25 21.0
  * longitude               (longitude) float32 36B 113.0 113.2 ... 114.8 115.0
  * init_time_utc           (init_time_utc) datetime64[ns] 416B 2021-09-26 .....
Dimensions without coordinates: ntb
Data variables: (12/22)
    valid_date_time0        (init_time_utc, step, time0) |S10 27kB b'20211010...
    ref_date_time0          (init_time_utc, step, time0) |S10 27kB b'20210924...
    forecast_hour0          (init_time_utc, step, time0) timedelta64[ns] 22kB ...
    time_bnds1              (init_time_utc, step, time1, ntb) datetime64[ns] 43kB ...
    valid_date_time_range1  (init_time_utc, step, time1, ntb) |S10 54kB b'202...
    ref_date_time1          (init_time_utc, step, time1) |S10 27kB b'20210924...
    ...                      ...
    M_CDC_L224_Avg_1        (init_time_utc, step, latitude, longitude, time1) float32 876kB ...
    M_CDC_L224              (init_time_utc, step, latitude, longitude, time0) float32 876kB ...
    H_CDC_L234_Avg_1        (init_time_utc, step, latitude, longitude, time1) float32 876kB ...
    H_CDC_L234              (init_time_utc, step, latitude, longitude, time0) float32 876kB ...
    4LFTX_L1                (init_time_utc, step, latitude, longitude, time0) float32 876kB ...
    ALBDO_L1_Avg_1          (init_time_utc, step, latitude, longitude, time1) float32 876kB ...
Attributes:
    Creation date and time:  2024-10-09 02:35:43 -0600
    Conventions:             CF-1.5
    Creator:                 NCAR - CISL RDA (dattore)

In [30]:
time_selta_step_list-time_selta_step_list.min()

array([0.0000e+00, 2.1600e+13, 4.3200e+13, 6.4800e+13, 8.6400e+13,
       1.0800e+14, 1.2960e+14, 1.5120e+14, 1.7280e+14, 1.9440e+14,
       2.1600e+14, 2.3760e+14, 2.5920e+14, 2.8080e+14, 3.0240e+14,
       3.2400e+14, 3.4560e+14, 3.6720e+14, 3.8880e+14, 4.1040e+14,
       4.3200e+14, 4.5360e+14, 4.7520e+14, 4.9680e+14, 5.1840e+14,
       5.4000e+14, 5.6160e+14, 5.8320e+14, 6.0480e+14, 6.2640e+14,
       6.4800e+14, 6.6960e+14, 6.9120e+14, 7.1280e+14, 7.3440e+14,
       7.5600e+14, 7.7760e+14, 7.9920e+14, 8.2080e+14, 8.4240e+14,
       8.8560e+14, 9.2880e+14, 9.7200e+14, 1.0152e+15, 1.0584e+15,
       1.1016e+15, 1.1448e+15, 1.1880e+15, 1.2312e+15, 1.2744e+15,
       1.3176e+15, 1.3608e+15])

In [12]:
new_step_dim

<xarray.DataArray (step: 52)> Size: 416B
array([1.3824e+15, 1.3392e+15, 1.2960e+15, 1.2528e+15, 1.2096e+15,
       1.1664e+15, 1.1232e+15, 1.0800e+15, 1.0368e+15, 9.9360e+14,
       9.5040e+14, 9.0720e+14, 8.6400e+14, 8.4240e+14, 8.2080e+14,
       7.9920e+14, 7.7760e+14, 7.5600e+14, 7.3440e+14, 7.1280e+14,
       6.9120e+14, 6.6960e+14, 6.4800e+14, 6.2640e+14, 6.0480e+14,
       5.8320e+14, 5.6160e+14, 5.4000e+14, 5.1840e+14, 4.9680e+14,
       4.7520e+14, 4.5360e+14, 4.3200e+14, 4.1040e+14, 3.8880e+14,
       3.6720e+14, 3.4560e+14, 3.2400e+14, 3.0240e+14, 2.8080e+14,
       2.5920e+14, 2.3760e+14, 2.1600e+14, 1.9440e+14, 1.7280e+14,
       1.5120e+14, 1.2960e+14, 1.0800e+14, 8.6400e+13, 6.4800e+13,
       4.3200e+13, 2.1600e+13])
Dimensions without coordinates: step

In [9]:
ds_with_step 

<xarray.Dataset> Size: 67MB
Dimensions:                 (step: 52, init_time_utc: 52, time0: 1, time1: 1,
                             ntb: 2, latitude: 9, longitude: 9, level0: 41,
                             isobaricInhPa: 22)
Coordinates:
  * step                    (step) float64 416B 1.382e+15 1.339e+15 ... 2.16e+13
  * time0                   (time0) datetime64[ns] 8B 2021-10-10
  * time1                   (time1) datetime64[ns] 8B 2021-10-10
  * level0                  (level0) float32 164B 1e+03 975.0 ... 0.02 0.01
  * isobaricInhPa           (isobaricInhPa) float32 88B 1e+03 975.0 ... 50.0
  * latitude                (latitude) float32 36B 23.0 22.75 ... 21.25 21.0
  * longitude               (longitude) float32 36B 113.0 113.2 ... 114.8 115.0
  * init_time_utc           (init_time_utc) datetime64[ns] 416B 2021-09-26 .....
Dimensions without coordinates: ntb
Data variables: (12/22)
    valid_date_time0        (init_time_utc, step, time0) |S10 27kB b'20211010...
    ref_date_time0          (init_time_utc, step, time0) |S10 27kB b'20210924...
    forecast_hour0          (init_time_utc, step, time0) timedelta64[ns] 22kB ...
    time_bnds1              (init_time_utc, step, time1, ntb) datetime64[ns] 43kB ...
    valid_date_time_range1  (init_time_utc, step, time1, ntb) |S10 54kB b'202...
    ref_date_time1          (init_time_utc, step, time1) |S10 27kB b'20210924...
    ...                      ...
    M_CDC_L224_Avg_1        (init_time_utc, step, latitude, longitude, time1) float32 876kB ...
    M_CDC_L224              (init_time_utc, step, latitude, longitude, time0) float32 876kB ...
    H_CDC_L234_Avg_1        (init_time_utc, step, latitude, longitude, time1) float32 876kB ...
    H_CDC_L234              (init_time_utc, step, latitude, longitude, time0) float32 876kB ...
    4LFTX_L1                (init_time_utc, step, latitude, longitude, time0) float32 876kB ...
    ALBDO_L1_Avg_1          (init_time_utc, step, latitude, longitude, time1) float32 876kB ...
Attributes:
    Creation date and time:  2024-10-09 02:35:43 -0600
    Conventions:             CF-1.5
    Creator:                 NCAR - CISL RDA (dattore)

In [2]:
from ocf_datapipes.load.nwp.providers.utils import open_zarr_paths
import xarray as xr
zarr_path = '/home/ubuntu/junbin/PVnet_data_hk/nwp/final/gfs.0p25.2021101000.zarr'
gfs: xr.Dataset = open_zarr_paths(zarr_path, time_dim="init_time_utc")
print('gfs.dims', gfs.dims)
nwp: xr.DataArray = gfs.to_array()
nwp

gfs.dims FrozenMappingWarningOnValuesAccess({'init_time_utc': 52, 'step': 52, 'latitude': 9, 'longitude': 9, 'time0': 1, 'level0': 41, 'time1': 1, 'isobaricInhPa': 22, 'ntb': 2})


<xarray.DataArray (variable: 22, init_time_utc: 52, step: 52, latitude: 9,
                   longitude: 9, time0: 1, level0: 41, time1: 1,
                   isobaricInhPa: 22, ntb: 2)> Size: 70GB
dask.array<stack, shape=(22, 52, 52, 9, 9, 1, 41, 1, 22, 2), dtype=object, chunksize=(1, 52, 52, 9, 9, 1, 41, 1, 22, 2), chunktype=numpy.ndarray>
Coordinates:
  * init_time_utc  (init_time_utc) datetime64[ns] 416B 2021-09-26 ... 2021-10...
  * isobaricInhPa  (isobaricInhPa) float32 88B 1e+03 975.0 950.0 ... 100.0 50.0
  * latitude       (latitude) float32 36B 23.0 22.75 22.5 ... 21.5 21.25 21.0
  * level0         (level0) float32 164B 1e+03 975.0 950.0 ... 0.04 0.02 0.01
  * longitude      (longitude) float32 36B 113.0 113.2 113.5 ... 114.8 115.0
  * step           (step) float64 416B 1.382e+15 1.339e+15 ... 4.32e+13 2.16e+13
  * time0          (time0) datetime64[ns] 8B 2021-10-10
  * time1          (time1) datetime64[ns] 8B 2021-10-10
  * variable       (variable) object 176B '4LFTX_L1' ... 'valid_date_time_ran...
Dimensions without coordinates: ntb
Attributes:
    Conventions:             CF-1.5
    Creation date and time:  2024-10-09 02:35:43 -0600
    Creator:                 NCAR - CISL RDA (dattore)

### 这段代码可以测试nwp数据是否合格

In [124]:
from ocf_datapipes.load.nwp.providers.utils import open_zarr_paths
zarr_path = '/home/ubuntu/junbin/PVnet_data/NWP/20211010_00v2.zarr'
gfs: xr.Dataset = open_zarr_paths(zarr_path, time_dim="init_time_utc")
print('gfs.dims', gfs.dims)
nwp: xr.DataArray = gfs.to_array()
print('nwp.dims1', nwp.dims)

del gfs

nwp = nwp.rename({"variable": "channel"})

gfs.dims FrozenMappingWarningOnValuesAccess({'step': 93, 'latitude': 657, 'longitude': 1097, 'init_time_utc': 93, 'isobaricInhPa': 6})
nwp.dims1 ('variable', 'step', 'latitude', 'longitude', 'isobaricInhPa')


In [11]:
compared_nwp_zarr = '/home/ubuntu/junbin/PVnet_data/NWP/20211010_00v2.zarr'
ds_compare = xr.open_dataset(compared_nwp_zarr)
ds_compare


/home/ubuntu/anaconda3/envs/PVnet310v2/lib/python3.10/site-packages/xarray/backends/plugins.py:149: RuntimeWarning: 'h5netcdf' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/home/ubuntu/anaconda3/envs/PVnet310v2/lib/python3.10/site-packages/xarray/backends/plugins.py:149: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)


<xarray.Dataset> Size: 13GB
Dimensions:        (step: 93, latitude: 657, longitude: 1097,
                    init_time_utc: 93, isobaricInhPa: 6)
Coordinates:
  * init_time_utc  (init_time_utc) datetime64[ns] 744B 2021-10-10 ... 2021-10-15
  * isobaricInhPa  (isobaricInhPa) float64 48B 50.0 500.0 700.0 ... 950.0 1e+03
  * latitude       (latitude) float64 5kB 29.5 29.56 29.62 ... 70.38 70.44 70.5
  * longitude      (longitude) float64 9kB -23.5 -23.44 -23.38 ... 44.94 45.0
  * step           (step) timedelta64[ns] 744B 00:00:00 ... 5 days 00:00:00
    time           datetime64[ns] 8B ...
    valid_time     (step) datetime64[ns] 744B ...
Data variables: (12/27)
    alb_rad        (step, latitude, longitude) float32 268MB ...
    aswdifd_s      (step, latitude, longitude) float32 268MB ...
    aswdir_s       (step, latitude, longitude) float32 268MB ...
    cape_con       (step, latitude, longitude) float32 268MB ...
    clch           (step, latitude, longitude) float32 268MB ...
    clcl           (step, latitude, longitude) float32 268MB ...
    ...             ...
    v              (step, isobaricInhPa, latitude, longitude) float32 2GB ...
    v_10m          (step, latitude, longitude) float32 268MB ...
    vmax_10m       (step, latitude, longitude) float32 268MB ...
    w_snow         (step, latitude, longitude) float32 268MB ...
    ww             (step, latitude, longitude) float32 268MB ...
    z0             (step, latitude, longitude) float32 268MB ...
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             edzw
    GRIB_centreDescription:  Offenbach
    GRIB_edition:            2
    GRIB_subCentre:          255
    history:                 2023-08-31T18:22 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             Offenbach

In [52]:
ds_compare

<xarray.Dataset> Size: 13GB
Dimensions:        (step: 93, latitude: 657, longitude: 1097,
                    init_time_utc: 93, isobaricInhPa: 6)
Coordinates:
  * init_time_utc  (init_time_utc) datetime64[ns] 744B 2021-10-10 ... 2021-10-15
  * isobaricInhPa  (isobaricInhPa) float64 48B 50.0 500.0 700.0 ... 950.0 1e+03
  * latitude       (latitude) float64 5kB 29.5 29.56 29.62 ... 70.38 70.44 70.5
  * longitude      (longitude) float64 9kB -23.5 -23.44 -23.38 ... 44.94 45.0
  * step           (step) timedelta64[ns] 744B 00:00:00 ... 5 days 00:00:00
    time           datetime64[ns] 8B ...
    valid_time     (step) datetime64[ns] 744B ...
Data variables: (12/27)
    alb_rad        (step, latitude, longitude) float32 268MB ...
    aswdifd_s      (step, latitude, longitude) float32 268MB ...
    aswdir_s       (step, latitude, longitude) float32 268MB ...
    cape_con       (step, latitude, longitude) float32 268MB ...
    clch           (step, latitude, longitude) float32 268MB ...
    clcl           (step, latitude, longitude) float32 268MB ...
    ...             ...
    v              (step, isobaricInhPa, latitude, longitude) float32 2GB ...
    v_10m          (step, latitude, longitude) float32 268MB ...
    vmax_10m       (step, latitude, longitude) float32 268MB ...
    w_snow         (step, latitude, longitude) float32 268MB ...
    ww             (step, latitude, longitude) float32 268MB ...
    z0             (step, latitude, longitude) float32 268MB ...
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             edzw
    GRIB_centreDescription:  Offenbach
    GRIB_edition:            2
    GRIB_subCentre:          255
    history:                 2023-08-31T18:22 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             Offenbach

In [173]:
compared_nwp_zarr1_ori= '/home/ubuntu/junbin/PVnet_data/NWP/20211010_00.zarr'
ds_compare_ori = xr.open_dataset(compared_nwp_zarr1_ori)

/home/ubuntu/anaconda3/envs/PVnet310v2/lib/python3.10/site-packages/xarray/backends/plugins.py:149: RuntimeWarning: 'h5netcdf' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/home/ubuntu/anaconda3/envs/PVnet310v2/lib/python3.10/site-packages/xarray/backends/plugins.py:149: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)


In [174]:
ds_compare_ori

<xarray.Dataset> Size: 13GB
Dimensions:        (step: 93, latitude: 657, longitude: 1097, isobaricInhPa: 6)
Coordinates:
  * isobaricInhPa  (isobaricInhPa) float64 48B 50.0 500.0 700.0 ... 950.0 1e+03
  * latitude       (latitude) float64 5kB 29.5 29.56 29.62 ... 70.38 70.44 70.5
  * longitude      (longitude) float64 9kB -23.5 -23.44 -23.38 ... 44.94 45.0
  * step           (step) timedelta64[ns] 744B 00:00:00 ... 5 days 00:00:00
    time           datetime64[ns] 8B ...
    valid_time     (step) datetime64[ns] 744B ...
Data variables: (12/27)
    alb_rad        (step, latitude, longitude) float32 268MB ...
    aswdifd_s      (step, latitude, longitude) float32 268MB ...
    aswdir_s       (step, latitude, longitude) float32 268MB ...
    cape_con       (step, latitude, longitude) float32 268MB ...
    clch           (step, latitude, longitude) float32 268MB ...
    clcl           (step, latitude, longitude) float32 268MB ...
    ...             ...
    v              (step, isobaricInhPa, latitude, longitude) float32 2GB ...
    v_10m          (step, latitude, longitude) float32 268MB ...
    vmax_10m       (step, latitude, longitude) float32 268MB ...
    w_snow         (step, latitude, longitude) float32 268MB ...
    ww             (step, latitude, longitude) float32 268MB ...
    z0             (step, latitude, longitude) float32 268MB ...
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             edzw
    GRIB_centreDescription:  Offenbach
    GRIB_edition:            2
    GRIB_subCentre:          255
    history:                 2023-08-31T18:22 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             Offenbach

In [15]:
ds_with_step

<xarray.Dataset> Size: 67MB
Dimensions:                 (step: 52, init_time_utc: 52, time0: 1, time1: 1,
                             ntb: 2, latitude: 9, longitude: 9, level0: 41,
                             isobaricInhPa: 22)
Coordinates:
  * step                    (step) timedelta64[ns] 416B 16 days 00:00:00 ... ...
  * time0                   (time0) datetime64[ns] 8B 2021-10-10
  * time1                   (time1) datetime64[ns] 8B 2021-10-10
  * level0                  (level0) float32 164B 1e+03 975.0 ... 0.02 0.01
  * isobaricInhPa           (isobaricInhPa) float32 88B 1e+03 975.0 ... 50.0
  * latitude                (latitude) float32 36B 23.0 22.75 ... 21.25 21.0
  * longitude               (longitude) float32 36B 113.0 113.2 ... 114.8 115.0
  * init_time_utc           (init_time_utc) datetime64[ns] 416B 2021-09-26 .....
Dimensions without coordinates: ntb
Data variables: (12/22)
    valid_date_time0        (init_time_utc, step, time0) |S10 27kB b'20211010...
    ref_date_time0          (init_time_utc, step, time0) |S10 27kB b'20210924...
    forecast_hour0          (init_time_utc, step, time0) timedelta64[ns] 22kB ...
    time_bnds1              (init_time_utc, step, time1, ntb) datetime64[ns] 43kB ...
    valid_date_time_range1  (init_time_utc, step, time1, ntb) |S10 54kB b'202...
    ref_date_time1          (init_time_utc, step, time1) |S10 27kB b'20210924...
    ...                      ...
    M_CDC_L224_Avg_1        (init_time_utc, step, latitude, longitude, time1) float32 876kB ...
    M_CDC_L224              (init_time_utc, step, latitude, longitude, time0) float32 876kB ...
    H_CDC_L234_Avg_1        (init_time_utc, step, latitude, longitude, time1) float32 876kB ...
    H_CDC_L234              (init_time_utc, step, latitude, longitude, time0) float32 876kB ...
    4LFTX_L1                (init_time_utc, step, latitude, longitude, time0) float32 876kB ...
    ALBDO_L1_Avg_1          (init_time_utc, step, latitude, longitude, time1) float32 876kB ...
Attributes:
    Creation date and time:  2024-10-09 02:35:43 -0600
    Conventions:             CF-1.5
    Creator:                 NCAR - CISL RDA (dattore)

In [21]:
import pandas as pd

# 生成 target_times
target_times = pd.date_range(start='2021-10-10 04:00:00', end='2021-10-10 14:00:00', freq='H')

# 生成 selected_init_times，这里假设初始化时间是固定的，不会改变
selected_init_times = pd.to_datetime(['2021-10-10T00:00:00.000000000'] * len(target_times))

# 生成 steps，这里假设步长是每个小时
steps = pd.to_timedelta(['04:00:00', '05:00:00', '06:00:00', '07:00:00', '08:00:00', '09:00:00',
                         '10:00:00', '11:00:00', '12:00:00', '13:00:00', '14:00:00'])
target_times

/tmp/ipykernel_357180/3177505080.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  target_times = pd.date_range(start='2021-10-10 04:00:00', end='2021-10-10 14:00:00', freq='H')


DatetimeIndex(['2021-10-10 04:00:00', '2021-10-10 05:00:00',
               '2021-10-10 06:00:00', '2021-10-10 07:00:00',
               '2021-10-10 08:00:00', '2021-10-10 09:00:00',
               '2021-10-10 10:00:00', '2021-10-10 11:00:00',
               '2021-10-10 12:00:00', '2021-10-10 13:00:00',
               '2021-10-10 14:00:00'],
              dtype='datetime64[ns]', freq='h')

In [26]:
steps

TimedeltaIndex(['0 days 04:00:00', '0 days 05:00:00', '0 days 06:00:00',
                '0 days 07:00:00', '0 days 08:00:00', '0 days 09:00:00',
                '0 days 10:00:00', '0 days 11:00:00', '0 days 12:00:00',
                '0 days 13:00:00', '0 days 14:00:00'],
               dtype='timedelta64[ns]', freq=None)

In [28]:
ds_with_step.step

<xarray.DataArray 'step' (step: 52)> Size: 416B
array([  21600000000000,   43200000000000,   64800000000000,   86400000000000,
        108000000000000,  129600000000000,  151200000000000,  172800000000000,
        194400000000000,  216000000000000,  237600000000000,  259200000000000,
        280800000000000,  302400000000000,  324000000000000,  345600000000000,
        367200000000000,  388800000000000,  410400000000000,  432000000000000,
        453600000000000,  475200000000000,  496800000000000,  518400000000000,
        540000000000000,  561600000000000,  583200000000000,  604800000000000,
        626400000000000,  648000000000000,  669600000000000,  691200000000000,
        712800000000000,  734400000000000,  756000000000000,  777600000000000,
        799200000000000,  820800000000000,  842400000000000,  864000000000000,
        907200000000000,  950400000000000,  993600000000000, 1036800000000000,
       1080000000000000, 1123200000000000, 1166400000000000, 1209600000000000,
       1252800000000000, 1296000000000000, 1339200000000000, 1382400000000000],
      dtype='timedelta64[ns]')
Coordinates:
  * step     (step) timedelta64[ns] 416B 0 days 06:00:00 ... 16 days 00:00:00

In [25]:
xr_data = ds_with_step

coords = {"target_time_utc": target_times}
steps = target_times - selected_init_times
init_time_indexer = xr.DataArray(selected_init_times, coords=coords)
step_indexer = xr.DataArray(steps, coords=coords)
xr_data.sel(step=step_indexer)

KeyError: "not all values found in index 'step'"